We import `netgen.gui` to open the netgen GUI for displaying the ferromagnet. 
We then define the geometry, and draw the box. 
The refinement of the mesh is controlled by `H_MAX` in `ngmesh`.

In [97]:
from netgen.csg import *
from ngsolve import *
from ngsolve.utils import (
    Grad
)  # If I don't import these explicitly, VSCode reads them as missing.
import netgen.gui  # this opens up the netgen ui
import Magnetisation_Functions as magfunc
import Elastic_Functions as elfunc
import General_Functions as genfunc
import numpy as np
import scipy.sparse as sp
import time
T_MAX: float = 10  # The maximum time for the simulation
ALPHA: float = 5  # Dissipative constant in the LLG equation.
THETA: float = 0.51  # Should be strictly above 1/2 for unconditional stability
K: float = 0.01  # TIME STEP
KAPPA: float = 1.0  # Determines the relative strength of the elastic vs. magnetic parts.
H_MAX: float = 0.1  # Determines how fine the mesh should be.

In [98]:
def MakeGeometry():  # this makes a box, with labelled faces
    geometry = CSGeometry()
    left = Plane(Pnt(0, 0, 0), Vec(-1, 0, 0)).bc("left")
    right = Plane(Pnt(1, 1, 1), Vec(1, 0, 0)).bc("right")
    front = Plane(Pnt(0, 0, 0), Vec(0, -1, 0)).bc("front")
    back = Plane(Pnt(1, 1, 1), Vec(0, 1, 0)).bc("back")
    bot = Plane(Pnt(0, 0, 0), Vec(0, 0, -1)).bc("bot")
    top = Plane(Pnt(1, 1, 1), Vec(0, 0, 1)).bc("top")

    cube = left * right * front * back * bot * top
    geometry.Add(cube)
    # cube = OrthoBrick(Pnt(0,0,0), Pnt(1,1,1))
    geometry.Add(cube)
    return geometry


ngmesh = MakeGeometry().GenerateMesh(maxh=H_MAX)
# ngmesh.Save("cube.vol")
mesh = Mesh(ngmesh)
Draw(mesh)

In [99]:
fes_mag = VectorH1(
    mesh, order=1
)  # the finite element space for the magnetisation m_h^i
fes_eps_m = MatrixValued(
    H1(mesh, order=1), dim=3
)  # matrix FE space on the magnetic part
fes_disp = VectorH1(
    mesh, order=1, dirichlet="bot"
)  # the finite element space for the displacement u_h^i

# print(f"mag_ndof={fes_mag.ndof}, disp_ndof={fes_disp.ndof},\n, dispfree_ndof={fes_disp.FreeDofs()}")
mag_gfu = GridFunction(fes_mag)
disp_gfu = GridFunction(fes_disp)
# body force and traction force
body_factor = Parameter(-1.0)
f_body = CoefficientFunction( (0, 0, body_factor) )
surface_factor = Parameter(1)
g_surface = CoefficientFunction([(0,0,surface_factor) if bc=="left" else 0 for bc in mesh.GetBoundaries()])

In [100]:
# Initial conditions
mag_gfu = magfunc.give_random_magnetisation(mag_gfu)
disp_gfu = elfunc.give_random_displacement(disp_gfu)
velocity_gfu = elfunc.give_random_displacement(disp_gfu)  # An initial velocity. Should only be used once in iteration.
Draw(mag_gfu)
magfunc.magnetic_energy(mag_gfu, mesh)

309.5253514306897

In [101]:
mag_list = []
disp_list = []

In [102]:
num_steps = genfunc.ceiling_division(T_MAX, K)  # This is ceiling division, using upside-down floor division.
for i in range(num_steps):
    mag_gfu = magfunc.update_magnetisation(fes_mag, mag_gfu, fes_eps_m, ALPHA, THETA, K, KAPPA)
    Redraw()
    print(f"\nStep {i}:integral |m*m|dx = {Integrate(mag_gfu*mag_gfu, mesh, VOL)} (should be 1)")
    print(f"proj energy = {magfunc.projected_magnetic_energy(mag_gfu, mesh, fes_mag)}")
    print(f"energy = {magfunc.magnetic_energy(mag_gfu, mesh)}")


Step 0:integral |m*m|dx = 0.5492982049363235 (should be 1)
proj energy = 241.8984822062207
energy = 275.2149878180172

Step 1:integral |m*m|dx = 0.7325423959617193 (should be 1)
proj energy = 187.73169024763234
energy = 238.6232874521705

Step 2:integral |m*m|dx = 0.9170338972367952 (should be 1)
proj energy = 146.9642115707612
energy = 204.98808748664766

Step 3:integral |m*m|dx = 1.0850064626414175 (should be 1)
proj energy = 116.10509802635535
energy = 175.61646492699657

Step 4:integral |m*m|dx = 1.2250659022216657 (should be 1)
proj energy = 92.98521861082034
energy = 151.20363561816436

Step 5:integral |m*m|dx = 1.341363523275787 (should be 1)
proj energy = 75.4732612871449
energy = 130.66128219249907

Step 6:integral |m*m|dx = 1.4333088143428394 (should be 1)
proj energy = 62.53792370659566
energy = 114.31304682467982

Step 7:integral |m*m|dx = 1.5028171641884662 (should be 1)
proj energy = 53.117033265045634
energy = 101.79551884994075

Step 8:integral |m*m|dx = 1.555488493212